[![Open in Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/pantat88/segsmaker/blob/main/Segsmaker_ComfyUI.ipynb)

### Conda

In [ ]:
from IPython.display import HTML, display
import subprocess
def ass(command, message, color):
    display(HTML(f"<span style='color:{color};'>--- {message} ---</span>"))
    result = subprocess.run(command, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    display(HTML("<span style='color:cyan;'>Done</span>"))
ass('conda install -q -y conda=23.10.0 glib aria2 gxx_linux-64 pyngrok ffmpeg av gst-libav imageio imageio-ffmpeg', 'Installing Conda', 'yellow')
ass('conda install -q -y -n base python=3.10.12', 'Installing Python 3.10.12', 'yellow')
ass('conda clean -y --all', 'Cleaning Conda environment', 'magenta')
!python --version
import os
os._exit(00)

### Clone ComfyUI

In [ ]:
from IPython.display import HTML, display
def ass(message, color):
    display(HTML(f"<span style='color:{color};'>{message}</span>"))
    
ass("Cloning ComfyUI. . .", "cyan")
%cd -q /home/studio-lab-user/
!git clone -q https://github.com/comfyanonymous/ComfyUI

ass("Installing requirements. . .", "yellow")
%cd -q ~/ComfyUI
!pip install https://download.pytorch.org/whl/cu118/xformers-0.0.22.post4%2Bcu118-cp310-cp310-manylinux2014_x86_64.whl -r requirements.txt torchvision --extra-index-url https://download.pytorch.org/whl/cu118 > /dev/null

ass("Creating a symlink to temporary folder. . .", "magenta")
!rm -rf ~/ComfyUI/models/checkpoints 
!rm -rf ~/ComfyUI/models/loras
!rm -rf ~/ComfyUI/models/controlnet
!rm -rf ~/tmp
!ln -vs /tmp ~/tmp
!ln -vs /tmp/checkpoints ~/ComfyUI/models/checkpoints
!ln -vs /tmp/loras ~/ComfyUI/models/loras
!ln -vs /tmp/controlnet_models ~/ComfyUI/models/controlnet
!rm -rf /home/studio-lab-user/.cache

ass("Done", "#66ff00")

### ComfyUI Custom Node

In [ ]:
from IPython.display import HTML, display
def ass(message, color):
    display(HTML(f"<span style='color:{color};'>{message}</span>"))
    
ass("Cloning ComfyUI Custom Node. . .", "cyan")
%cd -q ~/ComfyUI/custom_nodes

!git clone -q https://github.com/ssitu/ComfyUI_UltimateSDUpscale --recursive
!git clone -q https://github.com/ltdrdata/ComfyUI-Manager
!git clone -q https://github.com/pythongosssss/ComfyUI-Custom-Scripts
!git clone -q https://github.com/city96/SD-Latent-Upscaler
!git clone -q https://github.com/jags111/efficiency-nodes-comfyui
!git clone -q https://github.com/BlenderNeko/ComfyUI_ADV_CLIP_emb
!git clone -q https://github.com/ssitu/ComfyUI_NestedNodeBuilder
!git clone -q https://github.com/Kosinkadink/ComfyUI-Advanced-ControlNet
!git clone -q https://github.com/Fannovel16/comfyui_controlnet_aux
!git clone -q https://github.com/SLAPaper/ComfyUI-Image-Selector
!git clone -q https://github.com/Derfuu/Derfuu_ComfyUI_ModdedNodes
!git clone -q https://github.com/FizzleDorf/ComfyUI_FizzNodes
!git clone -q https://github.com/WASasquatch/was-node-suite-comfyui
!git clone -q https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite

!git clone -q https://github.com/Gourieff/comfyui-reactor-node
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/codeformer.pth  -d ~/ComfyUI/models/facerestore_models  -o codeformer.pth > /dev/null
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://github.com/TencentARC/GFPGAN/releases/download/v1.3.4/GFPGANv1.4.pth  -d ~/ComfyUI/models/facerestore_models  -o GFPGANv1.4.pth > /dev/null

!git clone -q https://github.com/Kosinkadink/ComfyUI-AnimateDiff-Evolved
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v15_v2.ckpt                  -d ~/ComfyUI/custom_nodes/ComfyUI-AnimateDiff-Evolved/models       -o mm_sd_v15_v2.ckpt > /dev/null
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_PanLeft.ckpt               -d ~/ComfyUI/custom_nodes/ComfyUI-AnimateDiff-Evolved/motion_lora  -o v2_lora_PanLeft.ckpt > /dev/null
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_PanRight.ckpt              -d ~/ComfyUI/custom_nodes/ComfyUI-AnimateDiff-Evolved/motion_lora  -o v2_lora_PanRight.ckpt > /dev/null
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_RollingAnticlockwise.ckpt  -d ~/ComfyUI/custom_nodes/ComfyUI-AnimateDiff-Evolved/motion_lora  -o v2_lora_RollingAnticlockwise.ckpt > /dev/null
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_RollingClockwise.ckpt      -d ~/ComfyUI/custom_nodes/ComfyUI-AnimateDiff-Evolved/motion_lora  -o v2_lora_RollingClockwise.ckpt > /dev/null
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_TiltDown.ckpt              -d ~/ComfyUI/custom_nodes/ComfyUI-AnimateDiff-Evolved/motion_lora  -o v2_lora_TiltDown.ckpt > /dev/null
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_TiltUp.ckpt                -d ~/ComfyUI/custom_nodes/ComfyUI-AnimateDiff-Evolved/motion_lora  -o v2_lora_TiltUp.ckpt > /dev/null
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_ZoomIn.ckpt                -d ~/ComfyUI/custom_nodes/ComfyUI-AnimateDiff-Evolved/motion_lora  -o v2_lora_ZoomIn.ckpt > /dev/null
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_ZoomOut.ckpt               -d ~/ComfyUI/custom_nodes/ComfyUI-AnimateDiff-Evolved/motion_lora  -o v2_lora_ZoomOut.ckpt > /dev/null

!git clone -q https://github.com/cubiq/ComfyUI_IPAdapter_plus
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15.safetensors            -d ~/ComfyUI/custom_nodes/ComfyUI_IPAdapter_plus/models  -o ip-adapter_sd15.safetensors > /dev/null
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15_light.safetensors      -d ~/ComfyUI/custom_nodes/ComfyUI_IPAdapter_plus/models  -o ip-adapter_sd15_light.safetensors > /dev/null
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus_sd15.safetensors       -d ~/ComfyUI/custom_nodes/ComfyUI_IPAdapter_plus/models  -o ip-adapter-plus_sd15.safetensors > /dev/null
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus-face_sd15.safetensors  -d ~/ComfyUI/custom_nodes/ComfyUI_IPAdapter_plus/models  -o ip-adapter-plus-face_sd15.safetensors > /dev/null
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-full-face_sd15.safetensors  -d ~/ComfyUI/custom_nodes/ComfyUI_IPAdapter_plus/models  -o ip-adapter-full-face_sd15.safetensors > /dev/null
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/image_encoder/model.safetensors        -d ~/ComfyUI/models/clip_vision                          -o 1-5-image-encoder.safetensors > /dev/null

ass("Installing Custom Node dependencies. . .", "yellow")
%cd -q ~/ComfyUI
!pip install opencv-python==4.7.0.72 transformers==4.26.1 tokenizers clip-interrogator > /dev/null
![ -f "custom_nodes/ComfyUI-Manager/scripts/colab-dependencies.py" ] && python "custom_nodes/ComfyUI-Manager/scripts/colab-dependencies.py" > /dev/null

!rm -rf /home/studio-lab-user/.cache
ass("Done", "#66ff00")
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

### ControlNET models
temporary

In [ ]:
!rm -rf ~/tmp/tmp && mkdir -p /tmp/controlnet_models
%cd -q ~/ComfyUI/models/controlnet

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors             -o control_v11e_sd15_ip2p_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11e_sd15_ip2p_fp16.yaml                        -o control_v11e_sd15_ip2p_fp16.yaml
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors          -o control_v11e_sd15_shuffle_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11e_sd15_shuffle_fp16.yaml                     -o control_v11e_sd15_shuffle_fp16.yaml
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny_fp16.safetensors            -o control_v11p_sd15_canny_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_canny_fp16.yaml                       -o control_v11p_sd15_canny_fp16.yaml
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors          -o control_v11f1p_sd15_depth_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11f1p_sd15_depth_fp16.yaml                     -o control_v11f1p_sd15_depth_fp16.yaml
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors          -o control_v11p_sd15_inpaint_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_inpaint_fp16.yaml                     -o control_v11p_sd15_inpaint_fp16.yaml
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart_fp16.safetensors          -o control_v11p_sd15_lineart_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_lineart_fp16.yaml                     -o control_v11p_sd15_lineart_fp16.yaml
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors             -o control_v11p_sd15_mlsd_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_mlsd_fp16.yaml                        -o control_v11p_sd15_mlsd_fp16.yaml
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors        -o control_v11p_sd15_normalbae_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_normalbae_fp16.yaml                   -o control_v11p_sd15_normalbae_fp16.yaml
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose_fp16.safetensors         -o control_v11p_sd15_openpose_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_openpose_fp16.yaml                    -o control_v11p_sd15_openpose_fp16.yaml
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble_fp16.safetensors         -o control_v11p_sd15_scribble_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_scribble_fp16.yaml                    -o control_v11p_sd15_scribble_fp16.yaml
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg_fp16.safetensors              -o control_v11p_sd15_seg_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_seg_fp16.yaml                         -o control_v11p_sd15_seg_fp16.yaml
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge_fp16.safetensors         -o control_v11p_sd15_softedge_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_softedge_fp16.yaml                    -o control_v11p_sd15_softedge_fp16.yaml
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors  -o control_v11p_sd15s2_lineart_anime_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15s2_lineart_anime_fp16.yaml             -o control_v11p_sd15s2_lineart_anime_fp16.yaml
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile_fp16.safetensors           -o control_v11f1e_sd15_tile_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11f1e_sd15_tile_fp16.yaml                      -o control_v11f1e_sd15_tile_fp16.yaml

### *Embeddings - Upscaler - VAE*

In [ ]:
# embeddings
%cd -q ~/ComfyUI/models/embeddings

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/pantat88/back_up/resolve/main/embeddings.zip -o embeddings.zip
!unzip -o ~/ComfyUI/models/embeddings/embeddings.zip
!rm embeddings.zip

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/150491 # edgQuality.pt

# Upscaler
%cd -q ~/ComfyUI/models/upscale_models

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/pantat88/ui/resolve/main/4x-UltraSharp.pth                        -o 4x-UltraSharp.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/pantat88/ui/resolve/main/4x-AnimeSharp.pth                        -o 4x-AnimeSharp.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/pantat88/ui/resolve/main/4x_RealisticRescaler_100000_G.pth        -o 4x_RealisticRescaler_100000_G.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/pantat88/ui/resolve/main/8x_RealESRGAN.pth                        -o 8x_RealESRGAN.pth

# VAE
%cd -q ~/ComfyUI/models/vae

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors                                                -o vae-ft-mse-840000-ema-pruned.safetensors
#aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/fp16-guy/anything_kl-f8-anime2_vae-ft-mse-840000-ema-pruned_blessed_clearvae_fp16_cleaned/resolve/main/kl-f8-anime2_fp16.safetensors    -o kl-f8-anime2_fp16.safetensors
#aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/fp16-guy/anything_kl-f8-anime2_vae-ft-mse-840000-ema-pruned_blessed_clearvae_fp16_cleaned/resolve/main/blessed2_fp16.safetensors        -o blessed2_fp16.safetensors
#aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/fp16-guy/anything_kl-f8-anime2_vae-ft-mse-840000-ema-pruned_blessed_clearvae_fp16_cleaned/resolve/main/ClearVAE_V2.3_fp16.safetensors   -o ClearVAE_V2.3_fp16.safetensors
#aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/fp16-guy/anything_kl-f8-anime2_vae-ft-mse-840000-ema-pruned_blessed_clearvae_fp16_cleaned/resolve/main/anything_fp16.safetensors        -o anything_fp16.safetensors


### *Checkpoint*
temporary

In [ ]:
!rm -rf ~/tmp/tmp && mkdir -p ~/tmp/checkpoints
%cd -q ~/ComfyUI/models/checkpoints

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/147913 -o meichidarkMixFp16_45.safetensors


### *Lora*
temporary

In [ ]:
!rm -rf ~/tmp/tmp && mkdir -p /tmp/loras
%cd -q ~/ComfyUI/models/loras

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/87153   -o add_detail.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/171989  -o detail_slider_v4.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/62833   -o Detail_Tweaker.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/86247   -o flat2.safetensors


## Run ComfyUI

In [ ]:
ngrok_token="PASTE_TOKEN_HERE"

from threading import Timer
from queue import Queue
from pyngrok import ngrok

def ngrok_tunnel(port,queue,auth_token):
    ngrok.set_auth_token(auth_token)
    url = ngrok.connect(port)
    queue.put(url)

ngrok_output_queue = Queue()
ngrok_thread = Timer(2, ngrok_tunnel, args=(8188, ngrok_output_queue, ngrok_token))
ngrok_thread.start()
ngrok_thread.join()
print(ngrok_output_queue.get())

%cd -q ~/ComfyUI
!python main.py --dont-print-server

### Output images
compress output into ```zip``` 

In [ ]:
input_folder  = '/home/studio-lab-user/ComfyUI/output'
output_folder = '/home/studio-lab-user'

import os
import zipfile
from tqdm.auto import tqdm
def zip_folder(input_folder, output_folder, max_size_mb=20):
    os.makedirs(output_folder, exist_ok=True)
    all_files = []
    for root, dirs, files in os.walk(input_folder):
        for file in files:
            file_path = os.path.join(root, file)
            all_files.append(file_path)
    zip_number = 1
    current_zip_size = 0
    current_zip_name = os.path.join(output_folder, f"part_{zip_number}.zip")
    with tqdm(total=len(all_files), desc='zipping : ', bar_format='{desc}{n_fmt}/{total_fmt} {bar} | {percentage:3.0f}% [ {elapsed}<{remaining}, {rate_fmt}{postfix} ]', ncols=700) as pbar:
        with zipfile.ZipFile(current_zip_name, 'w', zipfile.ZIP_DEFLATED) as current_zip:
            for file_path in all_files:
                file_size = os.path.getsize(file_path)
                if current_zip_size + file_size > max_size_mb * 1024 * 1024:
                    current_zip.close()
                    zip_number += 1
                    current_zip_name = os.path.join(output_folder, f"part_{zip_number}.zip")
                    current_zip = zipfile.ZipFile(current_zip_name, 'w', zipfile.ZIP_DEFLATED)
                    current_zip_size = 0
                current_zip.write(file_path, os.path.relpath(file_path, input_folder))
                current_zip_size += file_size
                pbar.update(1)
max_size_mb = 200
zip_folder(input_folder, output_folder, max_size_mb)

### For checking and cleaning Storage

In [ ]:
# Check Storage & List all folders

!df -h | grep -E 'Avail|home'
print ()
!du -h --max-depth=1 /home/studio-lab-user

In [ ]:
# DELETE output Images

!rm -rf /home/studio-lab-user/ComfyUI/output/*

In [ ]:
# DELETE ComfyUI

!unlink ~/tmp
!unlink ~/ComfyUI/models/checkpoints
!unlink ~/ComfyUI/models/loras
!unlink ~/ComfyUI/models/controlnet
!rm -rf /home/studio-lab-user/ComfyUI
!rm -rf /tmp/checkpoints /tmp/loras /tmp/controlnet_models
!rm -rf /home/studio-lab-user/tmp /home/studio-lab-user/.cache

In [ ]:
# DELETE EVERYTHING , restart jupyterlab afterward

!rm -rf /home/studio-lab-user/.cache
!rm -rf /home/studio-lab-user/.conda
!conda remove -n "env_name" --all
!rm -rf /home/studio-lab-user/*